# Objective:

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. WE are going to feed text excerpts from various books into a classifier and identify who is the author of the excerpts.


This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [2]:
#import NLP library abd english language model into Databricks
import spacy
spacy.cli.download('en')
nlp = spacy.load('en')

Linking successful 
 /local_disk0/pythonVirtualEnvDirs/virtualEnv-769d3b5a-2946-4c41-992e-a3bd496272ce/lib/python3.5/site-packages/en_core_web_sm
 --> /databricks/python/lib/python3.5/site-packages/spacy/data/en

 You can now load the model via spacy.load('en')

In [3]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType, FloatType, IntegerType
from pyspark.ml import Pipeline
from pyspark.ml.pipeline import Transformer
from pyspark.ml.feature import Tokenizer, StringIndexer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import CountVectorizer, HashingTF
from pyspark.ml.feature import IDF
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from spacy.lang.en import English

In [4]:
## books is  Spark Dataframe, hooray we can use SQL on this thing
books = spark.read.json('/FileStore/tables/guthnberg.json')

In [5]:
books.printSchema()

root
-- author: string (nullable = true)
-- excerpt: string (nullable = true)
-- title: string (nullable = true)

In [6]:
display(books.take(1))

author,excerpt,title
JaneAusten,"Chapter 1 It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife. However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters. “My dear Mr. Bennet,” said his lady to him one day, “have you heard that Netherfield Park is let at last?” Mr. Bennet replied that he had not. “But it is,” returned she; “for Mrs. Long has just been here, and she told me all about it.” Mr. Bennet made no answer. “Do you not want to know who has taken it?” cried his wife impatiently. “_You_ want to tell me, and I have no objection to hearing it.” This was invitation enough. “Why, my dear, you must know, Mrs. Long says that Netherfield is taken by a young man of large fortune from the north of England; that he came down on Monday in a chaise and four to see the place, and was so much delighted with it, that he agreed with Mr. Morris immediately; that he is to take possession before Michaelmas, and some of his servants are to be in the house by the end of next week.”",PrideAndPrejudice


In [7]:
#working with dataframe functions
books.groupby('author').count().show()

+--------------+-----+
 author|count|
+--------------+-----+
 MarkTwain| 2284|
 JohnMuir| 825|
 JaneAusten| 2378|
CharlesDickens| 3563|
+--------------+-----+

In [8]:
#create temporary tableview from DF to use SQL, which is only available to this notebook. You can also create table or globaltable in Spark.
books.createOrReplaceTempView("booksql")
booksql =spark.sql("Select author, count(*) as count from booksql group by author")
booksql.show()

+--------------+-----+
 author|count|
+--------------+-----+
 MarkTwain| 2284|
 JohnMuir| 825|
 JaneAusten| 2378|
CharlesDickens| 3563|
+--------------+-----+

In [9]:
%sql
Select author, count(*) as count from booksql group by author

author,count
MarkTwain,2284
JohnMuir,825
JaneAusten,2378
CharlesDickens,3563


# Process text data for machine learning

Unlike humans, machine can not understand languages. Hence we need to process text into vectors for machine learning.

We will do the following steps:
1. Tokenize: Split each sentence into list of individual words
2. TFIDF: create vector from split words, based on their occurences in that record and also how many records have those words.
3. feed these vectors into NaiveBayes classifier

##### NaiveBayes classifier is a "probabilistic classifiers" and tries to increase the probability that the record belongs to a particular class.

In [11]:
#This function used spacy tokenizer to split sentences into words
#Caution: UDF can not be optimized in Spark, so use them sparingly. UDF have to b registered with Spark UDF library.
#nlp = spacy.load('en')

def spacy_tokenize(text):
    global nlp
    return [text.lower_ for text in nlp(text)]

tokenize_udf = udf(lambda x: spacy_tokenize(x), ArrayType(StringType()))

In [12]:
books_token =books.withColumn('words', tokenize_udf(books.excerpt))
books_token.show(3)

+----------+--------------------+-----------------+--------------------+
 author| excerpt| title| words|
+----------+--------------------+-----------------+--------------------+
JaneAusten|Chapter 1 It is ...|PrideAndPrejudice|[chapter, 1, , i...|
JaneAusten|“What is his name...|PrideAndPrejudice|[“, what, is, his...|
JaneAusten|“In such cases, a...|PrideAndPrejudice|[“, in, such, cas...|
+----------+--------------------+-----------------+--------------------+
only showing top 3 rows

In [13]:
books_token.write.parquet("/FileStore/tables/tokenized_data")

In [14]:
books_token = spark.read.parquet("/FileStore/tables/tokenized_data")

In [15]:
# Set up transformers
author_labeler = StringIndexer(inputCol='author', outputCol='author_label')
tfvectorize= HashingTF(inputCol = 'words', outputCol = 'termfreq')
#countvec = CountVectorizer(inputCol='words', outputCol='termfreq')
idf = IDF(inputCol=tfvectorize.getOutputCol(), outputCol='tfidf')

In [16]:
%%time
# Now create the pipeline and build the dataframe by calling .fit() and .transform()
pipeline = Pipeline(stages=[author_labeler,tfvectorize, idf])
sample_data = pipeline.fit(books_token).transform(books_token)
sample_data.show(3)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-2537485322474769> in <module> () 
 ----> 1 get_ipython ( ) . run_cell_magic ( 'time' , '' , '# Now create the pipeline and build the dataframe by calling .fit() and .transform()\npipeline = Pipeline(stages=[author_labeler,tfvectorize, idf])\nsample_data = pipeline.fit(books_token).transform(books_token)\nsample_data.show(3)' ) 

 /databricks/python/lib/python3.5/site-packages/IPython/core/interactiveshell.py in run_cell_magic (self, magic_name, line, cell) 
 2160 magic_arg_s = self . var_expand ( line , stack_depth ) 
 2161 with self . builtin_trap : 
 -> 2162 result = fn ( magic_arg_s , cell ) 
 2163 return result 
 2164 

 <decorator-gen-60> in time (self, line, cell, local_ns) 

 /databricks/python/lib/python3.5/site-packages/IPython/core/magic.py in <lambda> (f, *a, **k) 
 191 # but it's overkill for just that one bit of state. 
 192 def magic_deco ( arg ) : 
 --> 193 call = lambda f , * a , ** k : f ( * a , ** k ) 
 194 
 195 if callable ( arg ) : 

 /databricks/python/lib/python3.5/site-packages/IPython/core/magics/execution.py in time (self, line, cell, local_ns) 
 1127 else : 
 1128 st = clock2 ( ) 
 -> 1129 exec ( code , glob , local_ns ) 
 1130 end = clock2 ( ) 
 1131 out = None 

 <timed exec> in <module> () 

 NameError : name 'books_token' is not defined

In [17]:
sample_data.printSchema()

root
-- author: string (nullable = true)
-- excerpt: string (nullable = true)
-- title: string (nullable = true)
-- words: array (nullable = true)
 |-- element: string (containsNull = true)
-- author_label: double (nullable = false)
-- termfreq: vector (nullable = true)
-- tfidf: vector (nullable = true)

In [18]:
%%time
# Save data frame
sample_data.write.parquet('/FileStore/tables/processed_data')

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-4075646379169512> in <module> () 
 ----> 1 get_ipython ( ) . run_cell_magic ( 'time' , '' , "# Save data frame\nsample_data.write.parquet('/FileStore/tables/processed_data')" ) 

 /databricks/python/lib/python3.5/site-packages/IPython/core/interactiveshell.py in run_cell_magic (self, magic_name, line, cell) 
 2160 magic_arg_s = self . var_expand ( line , stack_depth ) 
 2161 with self . builtin_trap : 
 -> 2162 result = fn ( magic_arg_s , cell ) 
 2163 return result 
 2164 

 <decorator-gen-60> in time (self, line, cell, local_ns) 

 /databricks/python/lib/python3.5/site-packages/IPython/core/magic.py in <lambda> (f, *a, **k) 
 191 # but it's overkill for just that one bit of state. 
 192 def magic_deco ( arg ) : 
 --> 193 call = lambda f , * a , ** k : f ( * a , ** k ) 
 194 
 195 if callable ( arg ) : 

 /databricks/python/lib/python3.5/site-packages/IPython/core/magics/execution.py in time (self, line, cell, local_ns) 
 1123 if mode == 'eval' : 
 1124 st = clock2 ( ) 
 -> 1125 out = eval ( code , glob , local_ns ) 
 1126 end = clock2 ( ) 
 1127 else : 

 <timed eval> in <module> () 

 NameError : name 'sample_data' is not defined

In [19]:
data = spark.read.parquet("/FileStore/tables/processed_data")

In [20]:
data.printSchema()

root
-- author: string (nullable = true)
-- excerpt: string (nullable = true)
-- title: string (nullable = true)
-- words: array (nullable = true)
 |-- element: string (containsNull = true)
-- author_label: double (nullable = true)
-- termfreq: vector (nullable = true)
-- tfidf: vector (nullable = true)

In [21]:
import pyspark as ps
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [22]:
%%time
splits = data.randomSplit(weights=[0.75, 0.25], seed=42)
train = splits[0]
test = splits[1]

train.persist()
test.persist()

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 24.3 ms

In [23]:
nb1 = NaiveBayes(smoothing=1.0, 
                 modelType='multinomial',
                 labelCol='author_label',
                 featuresCol='tfidf')
model1 = nb1.fit(train)

In [24]:
predictions = model1.transform(test)
predictions.persist()
print(predictions.printSchema())

root
-- author: string (nullable = true)
-- excerpt: string (nullable = true)
-- title: string (nullable = true)
-- words: array (nullable = true)
 |-- element: string (containsNull = true)
-- author_label: double (nullable = true)
-- termfreq: vector (nullable = true)
-- tfidf: vector (nullable = true)
-- rawPrediction: vector (nullable = true)
-- probability: vector (nullable = true)
-- prediction: double (nullable = false)

None

In [25]:
evaluator = MulticlassClassificationEvaluator(labelCol='author_label',
                                               predictionCol='prediction')

In [26]:
accuracy = evaluator.evaluate(predictions)
print('Author test set accuracy = ' + str(accuracy))

Author test set accuracy = 0.9906726381901833

In [27]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data] Unzipping tokenizers/punkt.zip.
 Out[ 9 ]: True

In [28]:
# File location and type
file_location = "/FileStore/tables/guthnberg.json"
file_type = "json"

# CSV options
infer_schema = "false"
first_row_is_header = "false"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

author,excerpt,title
JaneAusten,"Chapter 1 It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife. However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters. “My dear Mr. Bennet,” said his lady to him one day, “have you heard that Netherfield Park is let at last?” Mr. Bennet replied that he had not. “But it is,” returned she; “for Mrs. Long has just been here, and she told me all about it.” Mr. Bennet made no answer. “Do you not want to know who has taken it?” cried his wife impatiently. “_You_ want to tell me, and I have no objection to hearing it.” This was invitation enough. “Why, my dear, you must know, Mrs. Long says that Netherfield is taken by a young man of large fortune from the north of England; that he came down on Monday in a chaise and four to see the place, and was so much delighted with it, that he agreed with Mr. Morris immediately; that he is to take possession before Michaelmas, and some of his servants are to be in the house by the end of next week.”",PrideAndPrejudice
JaneAusten,"“What is his name?” “Bingley.” “Is he married or single?” “Oh! Single, my dear, to be sure! A single man of large fortune; four or five thousand a year. What a fine thing for our girls!” “How so? How can it affect them?” “My dear Mr. Bennet,” replied his wife, “how can you be so tiresome! You must know that I am thinking of his marrying one of them.” “Is that his design in settling here?” “Design! Nonsense, how can you talk so! But it is very likely that he _may_ fall in love with one of them, and therefore you must visit him as soon as he comes.” “I see no occasion for that. You and the girls may go, or you may send them by themselves, which perhaps will be still better, for as you are as handsome as any of them, Mr. Bingley may like you the best of the party.” “My dear, you flatter me. I certainly _have_ had my share of beauty, but I do not pretend to be anything extraordinary now. When a woman has five grown-up daughters, she ought to give over thinking of her own beauty.”",PrideAndPrejudice
JaneAusten,"“In such cases, a woman has not often much beauty to think of.” “But, my dear, you must indeed go and see Mr. Bingley when he comes into the neighbourhood.” “It is more than I engage for, I assure you.” “But consider your daughters. Only think what an establishment it would be for one of them. Sir William and Lady Lucas are determined to go, merely on that account, for in general, you know, they visit no newcomers. Indeed you must go, for it will be impossible for _us_ to visit him if you do not.” “You are over-scrupulous, surely. I dare say Mr. Bingley will be very glad to see you; and I will send a few lines by you to assure him of my hearty consent to his marrying whichever he chooses of the girls; though I must throw in a good word for my little Lizzy.” “I desire you will do no such thing. Lizzy is not a bit better than the others; and I am sure she is not half so handsome as Jane, nor half so good-humoured as Lydia. But you are always giving _her_ the preference.” “They have none of them much to recommend them,” replied he; “they are all silly and ignorant like other girls; but Lizzy has something more of quickness than her sisters.”",PrideAndPrejudice
JaneAusten,"“Mr. Bennet, how _can_ you abuse your own children in such a way? You take delight in vexing me. You have no compassion for my poor nerves.” “You mistake me, my dear. I have a high respect for your nerves. They are my old friends. I have heard you mention them with consideration these last twenty years at least.” “Ah, you do not know what I suffer.” “But I hope you will get over it, and live to see many young men of four thousand a year come into the neighbourhood.” “It will be no use to us, if twenty such sh

In [29]:
import pyspark.sql.functions as f
from pyspark.sql import types as t
#df = df.select('excerpt')
from collections import Counter
import string
import re

def my_map(x):
  #x = (x.replace('[^\w\s]', '').split())
  #regex = re.compile('[%s]' % re.escape(string.punctuation))
  #unpunctuated_text = regex.sub('', x) 
  return dict(Counter(x.split("")))

#map_udf = f.udf(my_map, t.StringType())
map_udf = f.udf(lambda x : my_map(x))

df = df.withColumn('line_split', map_udf(df['excerpt']))
df.show(10)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-2012848283785992> in <module> () 
 16 
 17 df = df . withColumn ( 'line_split' , map_udf ( df [ 'excerpt' ] ) ) 
 ---> 18 df . show ( 10 ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in show (self, n, truncate, vertical) 
 378 """
 379 if isinstance ( truncate , bool ) and truncate : 
 --> 380 print ( self . _jdf . showString ( n , 20 , vertical ) ) 
 381 else : 
 382 print ( self . _jdf . showString ( n , int ( truncate ) , vertical ) ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o3084.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 76.0 failed 1 times, most recent failure: Lost task 0.0 in stage 76.0 (TID 188, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
 File "/databricks/spark/python/pyspark/worker.py", line 377, in main
 process()
 File "/databricks/spark/python/pyspark/worker.py", line 372, in process
 serializer.dump_stream(func(split_index, iterator), outfile)
 File "/databricks/spark/python/pyspark/serializers.py", line 342, in dump_stream
 self.serializer.dump_stream(self._batched(iterator), stream)
 File "/databricks/spark/python/pyspark/serializers.py", line 141, in dump_stream
 for obj in iterator:
 File "/databricks/spark/python/pyspark/serializers.py", line 331, in _batched
 for item in iterator:
 File "<string>", line 1, in <lambda>
 File "/databricks/spark/python/pyspark/worker.py", line 85, in <lambda>
 return lambda *a: f(*a)
 File "/databricks/spark/python/pyspark/util.py", line 99, in wrapper
 return f(*args, **kwargs)
 File "<command-2012848283785992>", line 15, in <lambda>
 File "<command-2012848283785992>", line 12, in my_map
ValueError: empty separator

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:469)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:81)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:64)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:423)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$$anon$12.hasNext(Iterator.scala:439)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$11$$anon$1.hasNext(WholeStageCodegenExec.scala:619)
	at org.apache.spark.sql.execution.collect.UnsafeRowBatchUtils$.encodeUnsafeRows(UnsafeRowBatchUtils.scala:51)
	at org.apache.spark.sql.execution.collect.Collector$$anonfun$2.apply(Collector.scala:148)
	at org.apache.spark.sql.execution.collect.Collector$$anonfun$2.appl

In [30]:
import pyspark.sql.functions as f
# split and explode are SQL functions
df = df.select(f.split(df['excerpt'], " ").alias('line_split'))
df = df.select(f.explode(df["line_split"]).alias("word")).where(f.length('word') > 0).groupby('word').count().orderBy('count', ascending=False)
df.show(10)

+----+------+
word| count|
+----+------+
 the|111093|
 and| 85948|
 of| 67824|
 to| 65362|
 a| 52273|
 I| 41015|
 in| 37908|
 was| 32527|
that| 26605|
 it| 20393|
+----+------+
only showing top 10 rows

In [31]:
f.split(df['excerpt'], " ")

Out[ 81 ]: Column<b'split(excerpt, )'>

In [32]:
# Create a view or table

guthenberg_books = "gunberg_json"

df.createOrReplaceTempView(guthenberg_books)

In [33]:
%sql

/* Query the created temp table in a SQL cell */

select excerpt from `guthnberg_json` WHERE author = 'MarkTwain'

excerpt
"CONTENTS. CHAPTER I. My Brother appointed Secretary of Nevada--I Envy His Prospective Adventures--Am Appointed Private Secretary Under Him--My Contentment Complete--Packed in One Hour--Dreams and Visions--On the Missouri River --A Bully Boat CHAPTER II. Arrive at St. Joseph--Only Twenty-five Pounds Baggage Allowed--Farewell to Kid Gloves and Dress Coats--Armed to the Teeth--The “Allen”--A Cheerful Weapon--Persuaded to Buy a Mule--Schedule of Luxuries--We Leave the “States”--“Our Coach”--Mails for the Indians--Between a Wink and an Earthquake--A Modern Sphynx and How She Entertained Us--A Sociable Heifer CHAPTER III. “The Thoroughbrace is Broke”--Mails Delivered Properly--Sleeping Under Difficulties--A Jackass Rabbit Meditating, and on Business--A Modern Gulliver--Sage-brush--Overcoats as an Article of Diet--Sad Fate of a Camel--Warning to Experimenters CHAPTER IV. Making Our Bed--Assaults by the Unabridged--At a Station--Our Driver a Great and Shining Dignitary--Strange Place for a Frontyard --Accommodations--Double Portraits--An Heirloom--Our Worthy Landlord --“Fixings and Things”--An Exile--Slumgullion--A Well Furnished Table--The Landlord Astonished--Table Etiquette--Wild Mexican Mules--Stage-coaching and Railroading CHAPTER V. New Acquaintances--The Cayote--A Dog’s Experiences--A Disgusted Dog--The Relatives of the Cayote--Meals Taken Away from Home CHAPTER VI. The Division Superintendent--The Conductor--The Driver--One Hundred and Fifty Miles’ Drive Without Sleep--Teaching a Subordinate--Our Old Friend Jack and a Pilgrim--Ben Holliday Compared to Moses CHAPTER VII. Overland City--Crossing the Platte--Bemis’s Buffalo Hunt--Assault by a Buffalo--Bemis’s Horse Goes Crazy--An Impromptu Circus--A New Departure --Bemis Finds Refuge in a Tree--Escapes Finally by a Wonderful Method"
"CHAPTER VIII. The Pony Express--Fifty Miles Without Stopping--“Here he Comes”--Alkali Water--Riding an Avalanche--Indian Massacre CHAPTER IX. Among the Indians--An Unfair Advantage--Laying on our Arms--A Midnight Murder--Wrath of Outlaws--A Dangerous, yet Valuable Citizen CHAPTER X. History of Slade--A Proposed Fist-fight--Encounter with Jules--Paradise of Outlaws--Slade as Superintendent--As Executioner--A Doomed Whisky Seller--A Prisoner--A Wife’s Bravery--An Ancient Enemy Captured--Enjoying a Luxury--Hob-nobbing with Slade--Too Polite--A Happy Escape CHAPTER XI. Slade in Montana--“On a Spree”--In Court--Attack on a Judge--Arrest by the Vigilantes--Turn out of the Miners--Execution of Slade--Lamentations of His Wife--Was Slade a Coward? CHAPTER XII. A Mormon Emigrant Train--The Heart of the Rocky Mountains--Pure Saleratus--A Natural Ice-House--An Entire Inhabitant--In Sight of “Eternal Snow”--The South Pass--The Parting Streams--An Unreliable Letter Carrier--Meeting of Old Friends--A Spoiled Watermelon--Down the Mountain--A Scene of Desolation--Lost in the Dark--Unnecessary Advice --U.S. Troops and Indians--Sublime Spectacle--Another Delusion Dispelled --Among the Angels CHAPTER XIII. Mormons and Gentiles--Exhilarating Drink, and its Effect on Bemis--Salt Lake City--A Great Contrast--A Mormon Vagrant--Talk with a Saint--A Visit to the “King”--A Happy Simile CHAPTER XIV. Mormon Contractors--How Mr. Street Astonished Them--The Case Before Brigham Young, and How he Disposed of it--Polygamy Viewed from a New Position CHAPTER XV. A Gentile Den--Polygamy Discussed--Favorite Wife and D. 4--Hennery for Retired Wives--Children Need Marking--Cost of a Gift to No. 6 --A Penny-whistle Gift and its Effects--Fathering the Foundlings --It Resembled Him--The Family Bedstead"
"CHAPTER XVI The Mormon Bible--Proofs of its Divinity--Plagiarism of its Authors --Story of Nephi--Wonderful Battle--Kilkenny Cats Outdone CHAPTER XVII. Three Sides to all Questions--Everything “A Quarter”--Shriveled Up --Emigrants and White Shirts at a Discount--“Forty-Niners”--Above Par--Real Happiness CHAPTER XVIII. Alkali Desert--Romance of Crossing Dispelled--Alkali Dust--Effect 

In [34]:
# With this registered as a temp view, it will only be available to this particular notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.
# Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.
# To do so, choose your table name and uncomment the bottom line.

permanent_table_name = "guthnberg_json"

# df.write.format("parquet").saveAsTable(permanent_table_name)

In [35]:
import pyspark as ps 
sc

Out[ 6 ]: <SparkContext master=local[8] appName=Databricks Shell>

In [36]:
#procssing data to feed into the classifier
books = books.withColumn('words', F.split(books['excerpt'], " "))
books.show()